In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver

import re
import time

# 다이닝코드

In [2]:
D_restaurants_df = pd.DataFrame(columns=['name','address','category','main_mn','price','opng_tm','rating','rvw_cnt','tags'])
D_review_df = pd.DataFrame(columns=['res_name','user_name','rating','review'])

In [3]:
with open("../data/restaurants_DiningCode.txt","r") as f:
    text = f.read()
    urls = text.split()

In [4]:
urls

['https://www.diningcode.com/profile.php?rid=gymPpxy0wqyW',
 'https://www.diningcode.com/profile.php?rid=oFt7VQgJPD1x',
 'https://www.diningcode.com/profile.php?rid=rBGa94tjOhim',
 'https://www.diningcode.com/profile.php?rid=BIEVntjGgPBR',
 'https://www.diningcode.com/profile.php?rid=qYdn9yfYsqqR',
 'https://www.diningcode.com/profile.php?rid=ZbmKyMjf8LyV',
 'https://www.diningcode.com/profile.php?rid=X1677Q73zrTn',
 'https://www.diningcode.com/profile.php?rid=3m2pRph0pdjc',
 'https://www.diningcode.com/profile.php?rid=4pdt1C6gTjcU',
 'https://www.diningcode.com/profile.php?rid=xqY9zju9W4iZ',
 'https://www.diningcode.com/profile.php?rid=moKeujGlfTTW',
 'https://www.diningcode.com/profile.php?rid=GcNxur2JKeiv',
 'https://www.diningcode.com/profile.php?rid=WEh6gjz1tVd6',
 'https://www.diningcode.com/profile.php?rid=WIw4lV7nuN5O',
 'https://www.diningcode.com/profile.php?rid=9TUCHgdvAGHI',
 'https://www.diningcode.com/profile.php?rid=RPQbcSFyplWY',
 'https://www.diningcode.com/profile.php

In [5]:
def crawling_reviews_DiningCode(name, link):
    reviews = pd.DataFrame(columns=['res_name','user_name','rating','review'])
    try:
        driver.get(link)
    except:
        return None
    
    while True:
        try:
            click_more = driver.find_element_by_xpath("""//*[@id="div_more_review"]""")
            click_more.click()
        except:
            break
            
    time.sleep(1)
    
    try:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        for user in soup.find('div',{'id':'div_review'}).find_all('div',{'id':re.compile('div_review_+')}):
            tmp = user.find('span','btxt').get_text()
            user_name = tmp[:tmp.find('(')].strip()
            user_rating = int(re.findall('\d+',user.find('i',{'style':re.compile('width:+')})['style'])[0])//20
            user_review = user.find('p','review_contents btxt').get_text().replace('\n','')
        
            reviews = reviews.append({'res_name':name,
                                    'user_name':user_name,
                                    'rating':user_rating,
                                    'review':user_review}, ignore_index=True)
    except:
        return None
    
    driver.close()
    
    return reviews

In [6]:
D_restaurants_df

,name,address,category,main_mn,price,opng_tm,rating,rvw_cnt,tags


In [7]:
D_review_df

,res_name,user_name,rating,review


In [8]:
def crawling_res_data_DiningCode(link):
    
    #링크 불러오기
    try:
        html = urlopen(link)
        soup = BeautifulSoup(html, "html.parser")
    except:
        print('링크 에러:', link)
        return None,None,None,None,None,None,None,None,None,None
    
    #상호명
    try:
        name=soup.find_all('p','tit')[3].get_text()
        print(name)
    except:
        name=None
    
    #주소
    try:
        address = soup.find('li','locat').get_text()
    except:
        address = None
    
    #업종(다이닝코드엔 업종이 없음)
    category = None
    
    #주메뉴
    try:
        main_menu = soup.find('div','menu-info short').find_all('p','l-txt')[0].get_text().strip()
    except:
        main_menu = None
    
    #가격(주메뉴)
    try:
        price = int(soup.find('div','menu-info short').find_all('p','r-txt')[0].get_text()[:-1].replace(',',''))
    except:
        price = None
    
    #영업시간
    try:
        time_dict = dict()
        opening_time_list = soup.find('div','busi-hours short').find_all('p')[1:]
        for i in range(len(opening_time_list))[0::2]:
            if opening_time_list[i].get_text()!='더보기':
                time_dict[opening_time_list[i].get_text()] = opening_time_list[i+1].get_text()
    
        opening_time = str(time_dict)
    except:
        opening_time = None
    
    #평점과 리뷰 수
    try:
        ratings = float(soup.find('strong','rate-point').span.get_text())
        tmp = ratings[0].get_text().strip()
        rating = float(ratings[1].get_text()[:-1])
        review_count = tmp[:tmp.find('명')]
    except:
        rating = None
        review_count = None                 
    
    #태그
    try:
        tags = ','.join([i.get_text()[:i.get_text().find('(')] for i in soup.find_all('p','icon')])
    except:
        tags = None
    
    res_reviews = crawling_reviews_DiningCode(name, url)
    
    return name, address, category, main_menu, price, opening_time, rating, review_count, tags, res_reviews

In [9]:
count = 1

for url in urls:
    driver = webdriver.Chrome('../driver/chromedriver')

    print(count,'\t|',end='')
    
    name, address, category, main_menu, price, opening_time, rating, review_count, tags, review_df = crawling_res_data_DiningCode(url)    
    D_restaurants_df = D_restaurants_df.append({'name':name,
                                            'address':address,
                                            'category':category,
                                            'main_mn':main_menu,
                                            'price':price,
                                            'opng_tm':opening_time,
                                            'rating':rating,
                                            'rvw_cnt':review_count,
                                            'tags':tags}, ignore_index=True)

    D_review_df = D_review_df.append(review_df, ignore_index=True)

    count+=1

1 	|놀부만두
2 	|호반
3 	|민벅 건대점
4 	|뉴델리
5 	|강릉스낵
6 	|최고집칼국수
7 	|풀팬
8 	|여기가전집
9 	|벧엘제과
10 	|서일순대국
11 	|일성수산
12 	|오누이
13 	|대동문
14 	|낯선고기
15 	|샌드위치 하우스
16 	|홍차이 발산본점
17 	|손맛
18 	|브라더후드키친
19 	|다미
20 	|송정식당
21 	|미니회바
22 	|육회말연어
23 	|안다즈커피
24 	|공음
25 	|동해해물
26 	|라 플랑끄
27 	|느티나무설렁탕
28 	|우물우물
29 	|강릉엄지네 꼬막집
30 	|성천 막국수
31 	|퍼블리코 타코
32 	|은행골
33 	|일마지오 여의도점
34 	|돈불1971
35 	|데블스도어 본점
36 	|란주라미엔
37 	|코코로식당
38 	|다케롤
39 	|소문난식당
40 	|호남식당
41 	|타쿠미
42 	|강남불백 1호점
43 	|스시혼
44 	|맛있는집
45 	|머노까머나 신촌점
46 	|호 파스타
47 	|봉추찜닭
48 	|까망통돼지
49 	|몽탄
50 	|흥부전놀부전
51 	|열불날개
52 	|사모님돈가스
53 	|스타벅스
54 	|이몸이만든빵
55 	|파조쿠치나
56 	|여신족발
57 	|언니네 함바그
58 	|표준커피
59 	|순금이 떡볶이 아차산점
60 	|구복
61 	|우마이도 건대점
62 	|썸머타이
63 	|그집껍데기
64 	|서부감자국
65 	|기꾸스시
66 	|서북면옥
67 	|아리가또맘마
68 	|브레라
69 	|에헴
70 	|텐동 요츠야
71 	|등촌샤브칼국수 등촌점
72 	|콜린
73 	|로코스 비비큐 본점
74 	|부흥축산
75 	|에그썸
76 	|지구당
77 	|장독 흑돼지
78 	|기찻길연탄불생고기
79 	|매드포갈릭 타임스퀘어점
80 	|창고43 본점1관
81 	|온도
82 	|판데오로
83 	|무르무르 드 구스토
84 	|고냉지
85 	|남도묵은지
86 	|바다회포차
87 	|맥파이
88 	|훌리오
89 	|키에리
90 	|귀빈순대국
91 	|임가네 수제돈까스


669 	|평미가
670 	|망우 찜쌈밥
671 	|아웃백 천호점
672 	|가야가야
673 	|열불날개
674 	|주막
675 	|장가네
676 	|한나식빵 응암점
677 	|이차돌집
678 	|팬쿡
679 	|홍대돈부리 롯데월드몰점
680 	|닭도리탕19000원
681 	|펍피맥
682 	|맑은샘
683 	|봉선마라탕
684 	|깡통만두
685 	|메이다이닝
686 	|희정식당
687 	|원대구탕
688 	|아비꼬 여의도IFC점
689 	|신촌수제비
690 	|쭈꾸쭈꾸쭈꾸미
691 	|양미옥
692 	|마리웨일237 천호로데오점
693 	|성북동빵공장
694 	|참Soup
695 	|삼백돈 돈가츠
696 	|미시락
697 	|바오차이
698 	|프리모 바치오바치
699 	|수정궁
700 	|하동관 여의도점
701 	|그리지하우스
702 	|삼목등심
703 	|한솥도시락 가산디지털단지점
704 	|감성타코
705 	|토속촌
706 	|민들레식당
707 	|코스트코 푸드코트 상봉점
708 	|카니발피자 서울스퀘어점
709 	|순대국집
710 	|김수사
711 	|경주식당
712 	|메이탄
713 	|온더보더
714 	|브릭오븐
715 	|연어이야기
716 	|카츠젠 가산점
717 	|써브웨이
718 	|유림면
719 	|루엘드파리
720 	|소문난칼국수
721 	|자연별곡 목동점
722 	|나폴레옹과자점 본점
723 	|탄탄면공방 홍대점
724 	|메이크샐러드
725 	|신빛고을불고기
726 	|하이디라오 건대점
727 	|꼼떼
728 	|삼거리먼지막순대국
729 	|미소빵긋
730 	|샤이바나
731 	|구석집
732 	|앤트러사이트 한남점
733 	|페스타마레
734 	|메밀향그집
735 	|아빠곰탕
736 	|언니네함바그 회기점
737 	|마얘
738 	|란주탕슉
739 	|엉터리생고기
740 	|파스타농장
741 	|불스떡볶이
742 	|박용석스시 1호점
743 	|버터핑거 팬케이크
744 	|온파스타
745 	|을밀대
746 	|미즈컨테이너 롯데노원점

1305 	|민정식당
1306 	|브릭하우스76
1307 	|피스트
1308 	|종가집
1309 	|flot
1310 	|역시커피
1311 	|고명집
1312 	|온더보더 타임스퀘어점
1313 	|서가앤쿡 노원점
1314 	|이품분식
1315 	|스시혼
1316 	|가산통통
1317 	|충무칼국수
1318 	|순곱이네 건대점
1319 	|파스치노팬스테이크
1320 	|애성회관
1321 	|피오레37
1322 	|Jolly
1323 	|센타쯔
1324 	|폴앤폴리나
1325 	|매드포갈릭
1326 	|육영토종한우
1327 	|뽕나무쟁이 족발
1328 	|샘밭막국수
1329 	|고바우집
1330 	|백미당
1331 	|망향비빔국수
1332 	|훌리오 종로점
1333 	|햇빛촌 분점
1334 	|라티놀
1335 	|뭄알로이
1336 	|효동각
1337 	|24시전주콩나물국밥 쌍문점
1338 	|청화초밥
1339 	|사거리짬뽕집
1340 	|우부래도
1341 	|베러댄비프
1342 	|안동장
1343 	|조개깡패연구소
1344 	|손짱카레
1345 	|라임
1346 	|원조할매곱창
1347 	|딸랏롯빠이
1348 	|빠오즈푸
1349 	|태조 감자국
1350 	|만동제과
1351 	|삼오곱창
1352 	|금정
1353 	|리틀마나님
1354 	|옷살
1355 	|탐앤탐스
1356 	|고바우덩어리생고기
1357 	|이요이요스시
1358 	|신토불이떡볶이
1359 	|대관원
1360 	|곱창이야기 오목교점
1361 	|라구식당
1362 	|강동반상
1363 	|국시집
1364 	|서대문김치찜
1365 	|가마메시야 울림
1366 	|용초수
1367 	|엉터리생고기 가산디지털점
1368 	|논빼미
1369 	|곽만근갈비탕
1370 	|원조멸치국수 상봉점
1371 	|물음표
1372 	|복정집
1373 	|버드나무집
1374 	|사월식당
1375 	|한소반 청계산점
1376 	|구아이
1377 	|뎀셀브즈
1378 	|24시 우동집
1379 	|오레노라멘
1380 	|감초식

1908 	|연희동칼국수
1909 	|식탁의목적
1910 	|반포집
1911 	|MINNIM
1912 	|다케
1913 	|한우천국
1914 	|가장맛있는족발
1915 	|광주식당
1916 	|애슐리 가산점
1917 	|정식당
1918 	|통로
1919 	|나베식당
1920 	|어스핸드위치
1921 	|조금
1922 	|소담촌 노원역점
1923 	|선유기지
1924 	|이화
1925 	|데판야끼 한
1926 	|이디야 수유1동점
1927 	|구이혁명가 철든놈 제2공장
1928 	|통인스윗
1929 	|로꼬로꼬 영등포점
1930 	|웰빙멸치국수
1931 	|농민백암순대 본점
1932 	|구루메라멘
1933 	|시드누아
1934 	|퍼블릭맨션
1935 	|뿜뿌이
1936 	|더브라운
1937 	|아이엠어버거
1938 	|마굿간생고기
1939 	|사월에 보리밥과 쭈꾸미 현대디큐브신도림점
1940 	|황실짜장
1941 	|마렘마
1942 	|롤링스윗츠
1943 	|돌배기집
1944 	|599버거
1945 	|이치류 홍대본점
1946 	|목동분식
1947 	|카페 다담 수유점
1948 	|창신동매운족발 석계점
1949 	|맛닭꼬
1950 	|베나레스
1951 	|로리스 더프라임 립
1952 	|부띠끄경성
1953 	|와가리피순대
1954 	|평양면옥 논현점
1955 	|신호닭발 본점
1956 	|쉐프조
1957 	|피자아일랜드
1958 	|청진옥
1959 	|자코비버거
1960 	|달콤한 거짓말
1961 	|도나도나
1962 	|다래함박스텍
1963 	|코메또히또
1964 	|썬댄스플레이스
1965 	|명동칼국수
1966 	|카페 뤽상부르
1967 	|스타벅스 천호로데오점
1968 	|더플레이스
1969 	|자주테이블
1970 	|삼양여관
1971 	|꽃피는초밥
1972 	|애플하우스
1973 	|쿠마키친
1974 	|해적
1975 	|파머스빈 가산직영점
1976 	|손여사
1977 	|라면 땡기는 날
1978 	|도스타코스 중앙대학점
1979 	|역전우동0410 연신

In [10]:
D_restaurants_df.head()

,name,address,category,main_mn,price,opng_tm,rating,rvw_cnt,tags
0,놀부만두,서울특별시 동대문구 휘경2동 276-33,None,냉모밀,6000,"{'월-금,일': '오전 11시 - 오후 8시 30분 '}",None,None,"점심식사,혼밥,저녁식사,간식,지역주민이 찾는,서민적인,시끌벅적한,가성비좋은,푸짐한,..."
1,호반,서울특별시 종로구 낙원동 85-7,None,순대국,8000,{'매일': '오전 11시 30분 - 오후 10시 (준비시간 15:00~16:30)...,None,None,"저녁식사,점심식사,식사모임,가족외식,술모임,접대,혼술,데이트,혼밥,회식,아침식사,서..."
2,민벅 건대점,서울특별시 광진구 화양동 10-1,None,알리오올리오,9900,{'매일': '오전 11시 30분 - 오후 10시 (구정 추석 당일 3시 오픈)'},None,None,"점심식사,저녁식사,데이트,여자끼리,가족외식,모임,기념일,식사모임,아침식사,이국적/이..."
3,뉴델리,서울시 동대문구 회기동 3-182,None,치킨 탄두리 (한마리),15000,{'매일': '오전 11시 - 오후 10시 '},None,None,"점심식사,저녁식사,데이트,식사모임,이국적/이색적,깔끔한,캐주얼한"
4,강릉스낵,서울특별시 양천구 목동 406-65,None,직송 모듬 회,60000,"{'월-금': '오후 5시 - 오전 1시 ', '토/일': '오후 4시 - 오전 1...",None,None,"술모임,점심식사,저녁식사,간식,데이트,아이동반,가족외식,식사모임,혼밥,서민적인,캐주..."


In [11]:
D_review_df

,res_name,user_name,rating,review
0,놀부만두,Youngmi Lee,5,지역맛집답게 적당히 허름하고 작지만 손님이 꽉차 있었다. 냉모밀 국물은 깊은달달함~...
1,놀부만두,Kunwoo Kim,4,제기동 평양냉면 갔다가 정기휴일 발길돌려휘경동 생방송투데이 맛집 놀부만두로 아버지와...
2,놀부만두,erica,3,비빔모밀 온모밀 김치만두 먹어봄유명세에 비해 그닥...그나마 셋 중에는 김치만두가 ...
3,놀부만두,발챙이,4,김치만두랑 냉모밀 조합이 최고입니다. 만두피가 엄청 쫄깃해요.
4,놀부만두,nelia,5,
5,호반,검정치마,4,익선동 골목 끄트머리에 자리잡은 한식집입니다. 내장순대가 냄새없이 맛있고 우거지탕도...
6,호반,꼬봉도령,5,고루 맛잇는 내공잇는 맛집. 서비스 좋으세요. 흔햇던 메뉴도 나름의 색다른 맛이 잇...
7,호반,lindachung,4,나 홀로 순대국(7000원) 을 시키면 나오는 반찬 구성은 물김치와 꽁치 한토막 그...
8,호반,ysp,4,음식이 다 맛있네요!!시그니처 병어조림과 순대. 낙지볶음.금액대가 좀 부담스럽긴 합...
9,호반,김영석,4,아침에 가서 그런지 조용했다 일단 맛이 좋다 만두같은 순대에 큼직한 순대가 들어간 ...


In [12]:
D_restaurants_df.to_csv('../data/DiningCode_df.csv',index=False)
D_review_df.to_csv('../data/DiningCode_review_df.csv', index=False)

# 망고플레이트

In [13]:
M_restaurants_df = pd.DataFrame(columns=['name','address','category','main_mn','price','opng_tm','rating','rvw_cnt','tags'])
M_review_df = pd.DataFrame(columns=['res_name','user_name','rating','review'])

In [14]:
with open("../data/restaurants_MangoPlate.txt","r") as f:
    text = f.read()
    Mango_urls = text.split()

In [15]:
Mango_urls

['https://www.mangoplate.com/restaurants/AzUTJfcERz7_',
 'https://www.mangoplate.com/restaurants/2vx89FA1cBrE',
 'https://www.mangoplate.com/restaurants/xU-GrrZeGYSf',
 'https://www.mangoplate.com/restaurants/FN7e6CAusN',
 'https://www.mangoplate.com/restaurants/WX16nZzHAdgU',
 'https://www.mangoplate.com/restaurants/At5HxAIOYwzS',
 'https://www.mangoplate.com/restaurants/ASMtBoYXTc1z',
 'https://www.mangoplate.com/restaurants/8JU-3Qnr0M',
 'https://www.mangoplate.com/restaurants/FT6X0FrSqOx9',
 'https://www.mangoplate.com/restaurants/XgdJPVEYZxDW',
 'https://www.mangoplate.com/restaurants/8BZ8SYaOHBJA',
 'https://www.mangoplate.com/restaurants/TOi0JORtQs',
 'https://www.mangoplate.com/restaurants/KYgQhlRY0r',
 'https://www.mangoplate.com/restaurants/pJAZTeYLDnfL',
 'https://www.mangoplate.com/restaurants/q1PDtSFDsXea',
 'https://www.mangoplate.com/restaurants/COs1JgXzb5',
 'https://www.mangoplate.com/restaurants/Uh8QK8fxlp69',
 'https://www.mangoplate.com/restaurants/X_e-O8s5PJrB',
 '

In [16]:
len(Mango_urls)

4279

In [17]:
def crawling_reviews_MangoPlate(name, link):
    reviews = pd.DataFrame(columns=['res_name','user_name','rating','review'])
    more_count = 0
    while (more_count<=20):
        try:
            time.sleep(1)
            driver.find_element_by_xpath("""/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]""").click()
            more_count+=1
        except:
            break

    tabs = driver.window_handles
    
    if len(tabs)>=2:
        for i in range(len(tabs))[:0:-1]:
            driver.switch_to.window(tabs[i])
            driver.close()

    driver.switch_to.window(tabs[0])
    
    try:
        time.sleep(0.5)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        users = soup.find_all('a','RestaurantReviewItem__Link')

        rating_score = {'맛있다':5,'괜찮다':3,'별로':1}
        for user in users:
            user_name = user.span.get_text()
            user_review = user.p.get_text().strip()
            user_rating = user.find('span','RestaurantReviewItem__RatingText').get_text()
            user_rating = rating_score[user_rating]
            
            reviews = reviews.append({'res_name':name,
                                    'user_name':user_name,
                                    'rating':user_rating,
                                    'review':user_review}, ignore_index=True)        
    except:
        return None
    
    
    return reviews

In [18]:
def crawling_res_data_MangoPlate(link):
    
    driver.get(link)
    #링크 불러오기
    try:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
    except:
        print('Link Error:', link)
        return None,None,None,None,None,None,None,None,None,None
    
    #상호명
    try:
        name= soup.find('h1','restaurant_name').get_text()
        print(name)
    except:
        name=None
    
    #주소
    try:
        address = soup.find('div','Restaurant__InfoItemLabel--RoadAddressText').get_text()
    except:
        address = None
    
    #업종
    try:
        category = soup.select('tbody > tr')[2].span.get_text()
    except:
        category = None
    
    #주메뉴
    try:
        main_menu = soup.find_all('span','Restaurant_Menu')[0].get_text()
    except:
        main_menu = None
    
    #가격(주메뉴)
    try:
        price = int(soup.find_all('span','Restaurant_MenuPrice')[0].get_text()[:-1].replace(',',''))
    except:
        price = None
    
    #영업시간
    try:
        opening_time = soup.select('tbody > tr')[5].td.get_text()
    except:
        opening_time = None
    
    #평점 
    try:
        rating = float(soup.find('strong','rate-point').span.get_text())
    except:
        rating = None
        
    #리뷰 수
    try:
        review_count = int(soup.find('span','cnt review').get_text())
    except:
        review_count = None                 
    
    #태그
    try:
        tags = ''
        for tag in soup.find('section','module related-tags only-desktop').find_all('a'):
            tags += tag.get_text().strip()
            tags += ','
    except:
        tags = None
    
    res_reviews = crawling_reviews_MangoPlate(name, link)
    
    return name, address, category, main_menu, price, opening_time, rating, review_count, tags, res_reviews

In [19]:
count = 1
driver = webdriver.Chrome('../driver/chromedriver')

for url in Mango_urls:
    print(count,'\t|',end='')
    
    name, address, category, main_menu, price, opening_time, rating, review_count, tags, review_df = crawling_res_data_MangoPlate(url)    
    M_restaurants_df = M_restaurants_df.append({'name':name,
                                            'address':address,
                                            'category':category,
                                            'main_mn':main_menu,
                                            'price':price,
                                            'opng_tm':opening_time,
                                            'rating':rating,
                                            'rvw_cnt':review_count,
                                            'tags':tags}, ignore_index=True)

    M_review_df = M_review_df.append(review_df, ignore_index=True)

    count+=1

1 	|제이든
2 	|본쁠리
3 	|카페 달력
4 	|라 쿠치나
5 	|사월식당
6 	|천진분식
7 	|EDEN
8 	|수정불막창
9 	|몬스터플레이스
10 	|카페미르
11 	|와레와레쿡
12 	|노량진할머니왕파전
13 	|스시박사
14 	|도우룸
15 	|소풍가는날
16 	|먼데이블루스
17 	|지노스뉴욕피자
18 	|삼정하누
19 	|황주집
20 	|또래오래
21 	|롱타임노씨
22 	|풀잎채
23 	|삼미제면
24 	|더은교
25 	|유성관
26 	|미분당
27 	|고기파티
28 	|쌍다리돼지불백
29 	|누하동 주스바
30 	|세상의모든아침
31 	|백만불식품
32 	|진미각
33 	|미완성식탁
34 	|뽕족
35 	|모소리
36 	|쁘띠크로아상
37 	|크래프트한스
38 	|대한닭발1979
39 	|창신동매운족발
40 	|방학동이야기
41 	|대우부대찌개
42 	|초밥좋은날
43 	|메이크샐러드
44 	|유유자적
45 	|노다지
46 	|갈리나데이지
47 	|모이리타
48 	|초류향
49 	|유리연구소
50 	|스시이마
51 	|윤씨네족보
52 	|스시다네
53 	|제뉴어리피크닉
54 	|칸텐
55 	|레드어니언
56 	|삼숙이갈비
57 	|김여사네국수
58 	|헤비로테이트
59 	|올댓피자
60 	|등촌샤브칼국수
61 	|태양커피
62 	|포데이
63 	|마실
64 	|인자칼국수
65 	|허유산
66 	|미분당
67 	|누하의숲
68 	|허브족발
69 	|돈짱맛짱
70 	|외고집설렁탕
71 	|오헨
72 	|억조
73 	|알촌
74 	|커피바이
75 	|마포족발순대국
76 	|도원
77 	|고호비
78 	|레더라
79 	|생고기삼백
80 	|대만야시장포차
81 	|개인공간
82 	|버터밀크
83 	|모티프커피바
84 	|미즈컨테이너
85 	|구이명가
86 	|누들아한타이
87 	|르미엘
88 	|파술타
89 	|미미초밥
90 	|고도식
91 	|대박전집
92 	|애성회관
93 	|릿잇타미
94 	|일월생
95 	|이와타
96 	|킷사서울
97 	|

704 	|차녕식당
705 	|하영족발과보쌈
706 	|소백양샤브샤브
707 	|카페브릭
708 	|아웃백스테이크하우스
709 	|제일콩집
710 	|고씨네
711 	|봉샌드
712 	|연어시장
713 	|커피별녹색잔
714 	|파티엘하우스
715 	|남춘식당
716 	|대영식당
717 	|더제이케이키친박스
718 	|더페이머스램
719 	|엄마손칼국수
720 	|별버거
721 	|타마고
722 	|테르미니
723 	|파낙스
724 	|애슐리
725 	|프롬투커피
726 	|메이탄
727 	|오아저씨수제고로케
728 	|버거파크
729 	|평양면옥
730 	|밀탑
731 	|청수장
732 	|본가
733 	|목란
734 	|일도씨닭갈비
735 	|채선당
736 	|마루기
737 	|아씨펍
738 	|카페고르
739 	|비어바나
740 	|리도
741 	|웨일펍 수제맥주
742 	|봉이만두
743 	|스담
744 	|크로네
745 	|베이커리봉교
746 	|알프리베
747 	|함스브로트과자점
748 	|카페오픈더도어
749 	|강화통통생고기
750 	|아우어베이커리
751 	|십분의일
752 	|삼정
753 	|아이엠부대찌개
754 	|탁초밥
755 	|착한피자
756 	|쿠차라
757 	|나인온스버거
758 	|제주아방
759 	|파르크
760 	|하루돈까스
761 	|4TENTIAL
762 	|춘천골
763 	|요요스시
764 	|모퉁이우
765 	|더메뉴얼
766 	|치즈룸
767 	|다전식당
768 	|BISTROT de YOUNTVILLE
769 	|도봉관
770 	|원할머니보쌈족발
771 	|우뇽파스타
772 	|깻잎치킨
773 	|구전동화
774 	|커피매터스
775 	|백마 김씨네
776 	|루프탑바플로팅
777 	|더베리와플
778 	|호박
779 	|고대맛집ㅋㅋ
780 	|대흥양다리바베큐
781 	|스와가트
782 	|체부동잔치집
783 	|유즈라멘
784 	|뱃놈
785 	|피크니크
786 	|투고샐러드
787 	|호계식
788 	|고냉지
7

1370 	|포보스
1371 	|목동아구찜
1372 	|아시아문
1373 	|로리스더프라임립
1374 	|메이다이닝
1375 	|아오리의행방불명
1376 	|미각
1377 	|Colors
1378 	|청키면가
1379 	|갈쿠리와소쿠리
1380 	|봉구스밥버거
1381 	|모나드커피로스터스
1382 	|포탈라
1383 	|꼬르소산도
1384 	|더 아래
1385 	|비엣꽌
1386 	|비로소커피
1387 	|감성당
1388 	|도치피자
1389 	|JVL부대찌개
1390 	|스시왕
1391 	|즉석바지락손칼국수
1392 	|더테이블
1393 	|성지곱창
1394 	|스시산원 경
1395 	|공휴일 카페
1396 	|장수돼지갈비
1397 	|프로즌빈카페
1398 	|와플앨리
1399 	|곰이순대국
1400 	|오월의커피
1401 	|식당진
1402 	|윤주방
1403 	|이치류
1404 	|루엘드파리
1405 	|모녀신떡&미쓰밀떡
1406 	|헌치브라운
1407 	|아방뮤제
1408 	|한우정성곱창
1409 	|충무호동복
1410 	|플라잉볼익스프레스
1411 	|독도수산
1412 	|후아나
1413 	|땅콩카페
1414 	|본가할머니보쌈
1415 	|나주곰탕냉면
1416 	|하정식당
1417 	|끼룩하우스
1418 	|신선설농탕
1419 	|몽탄
1420 	|애정마라샹궈
1421 	|가미우동
1422 	|그레이에스프레소
1423 	|좋은아침 PASTRY
1424 	|카페퍼블리코
1425 	|뽕나무쟁이
1426 	|홍대돈부리
1427 	|상남스시
1428 	|중경소면관
1429 	|더맛나곱창
1430 	|설눈
1431 	|까페온고
1432 	|라스커피
1433 	|노브랜드버거
1434 	|고가빈커리하우스
1435 	|스시범
1436 	|순이네고릴라
1437 	|강동호치민
1438 	|고씨네
1439 	|르방드지우
1440 	|슬로우치즈
1441 	|시타시이베이커리
1442 	|서울다이닝
1443 	|제주밀면촌
1444 	|종로제면소
1445 	|하남돼지집
144

2006 	|패스트리부티크
2007 	|강강술래
2008 	|와요
2009 	|삼육칠하우스
2010 	|육회랑연어랑
2011 	|후후스시
2012 	|장독대뼈다귀감자탕
2013 	|고을래
2014 	|커피디엔에이로스팅컴퍼니
2015 	|만두라
2016 	|오부장치킨
2017 	|아마르
2018 	|탕절대부
2019 	|막내회집
2020 	|김스시
2021 	|모범떡볶이와수제돈까스
2022 	|일미집
2023 	|은행골
2024 	|bowl deli and eatery
2025 	|알라프리마
2026 	|호호식당
2027 	|채선당
2028 	|장안정
2029 	|영원식당
2030 	|니뽕내뽕
2031 	|네기다이닝라운지
2032 	|참만나
2033 	|이파리
2034 	|주문진오징어보쌈
2035 	|국수나무
2036 	|재영통닭
2037 	|다독이네 숯불구이
2038 	|다꼬야끼
2039 	|비스트로친친
2040 	|어글리스토브
2041 	|남해횟집
2042 	|춘향골남원추어탕
2043 	|연희김밥
2044 	|채스우드커피
2045 	|파우치커피
2046 	|그라놀로지
2047 	|대가방
2048 	|테스트키친
2049 	|미차이
2050 	|베브릿지
2051 	|인우초밥
2052 	|할랄가이즈
2053 	|트웰브
2054 	|프라이밋 스테이크하우스
2055 	|신천샤브칼국수
2056 	|종로곱육개장
2057 	|공릉곱창
2058 	|스트라다로스터스
2059 	|샛별제과
2060 	|정담은보쌈
2061 	|커피기업
2062 	|동흥관
2063 	|삿뽀로
2064 	|노원소곱창
2065 	|홈프롬귀
2066 	|몽상가 크레이프 전문점
2067 	|하노이별
2068 	|카페백야
2069 	|파스토
2070 	|장수살롱
2071 	|갓포아키
2072 	|한참치
2073 	|북촌손만두
2074 	|천년곱돌
2075 	|하이제씨
2076 	|던모스
2077 	|콩이네
2078 	|땡초불닭발
2079 	|혜미
2080 	|송림식당
2081 	|발해양꼬치
2082 	|꽃다

2644 	|채선당
2645 	|원성반점
2646 	|주대국초밥집
2647 	|헝그리서울
2648 	|매일이 기념일
2649 	|짚신매운갈비찜
2650 	|채선당
2651 	|떼레노
2652 	|무르무르드마르젬
2653 	|제로컴플렉스
2654 	|경이로운
2655 	|호천식당
2656 	|곱창나라
2657 	|카페306
2658 	|육촌
2659 	|당일왕족발
2660 	|경양미식
2661 	|시월애도토리
2662 	|카페드람브르
2663 	|스시코우지
2664 	|소금집델리 안국
2665 	|WE Ccafe
2666 	|다루다
2667 	|항아리닭갈비막국수
2668 	|횟집울릉도
2669 	|스시산
2670 	|봉평고향메밀촌
2671 	|의정부평양면옥
2672 	|미가양꼬치
2673 	|차칸스시
2674 	|시찌린
2675 	|코코도리
2676 	|뉴타운
2677 	|옛날마차
2678 	|원차우
2679 	|코인
2680 	|성창식당
2681 	|아웃백스테이크하우스
2682 	|야사이마끼쿠이신보
2683 	|가빈커피로스터스
2684 	|치맥파이브
2685 	|Comfort Cafe
2686 	|쉘위커피
2687 	|설빙
2688 	|모미지쿡
2689 	|에머이
2690 	|카페그레도
2691 	|무삥과팟타이
2692 	|오코노미야키식당하나
2693 	|커피그라운즈
2694 	|유키가후루
2695 	|루틴
2696 	|이주소곱창
2697 	|우미로드뽕전골
2698 	|사이코우스시
2699 	|하이보
2700 	|결돈부리
2701 	|경성광장
2702 	|목바
2703 	|마초짬뽕
2704 	|망넛이네
2705 	|도치피자
2706 	|달콤한순간
2707 	|김가네짜장
2708 	|모이자포장마차로
2709 	|에이투지카페
2710 	|등촌미돈
2711 	|김덕후의곱창조
2712 	|바다애
2713 	|천사곱창
2714 	|빈스코드
2715 	|저팔계식당
2716 	|중화복춘 골드
2717 	|팻캣
2718 	|웨이브스
2719 	|유성집
2720 	|예지현
2

3288 	|오감부대찌개와철판볶음
3289 	|와인웍스
3290 	|포컵스
3291 	|옥천집
3292 	|범스피자
3293 	|은성장
3294 	|진짜해장국 대화정
3295 	|버거드조선
3296 	|존슨부대찌개
3297 	|멘무샤
3298 	|팬스테이크키친
3299 	|플라마카롱
3300 	|어비스
3301 	|춘천막국수
3302 	|고향집
3303 	|소친친
3304 	|장사랑
3305 	|막불감동
3306 	|레알라면
3307 	|젤라띠젤라띠
3308 	|채선당
3309 	|레프레미스
3310 	|너구리해장국
3311 	|마우스래빗
3312 	|거목정육식당
3313 	|가츠시
3314 	|만족오향족발
3315 	|플레잉디오븐
3316 	|풍천장어직판장
3317 	|경복궁
3318 	|오스테리아샘킴
3319 	|이다
3320 	|갓덴스시
3321 	|우리들숯불갈비
3322 	|맥파이
3323 	|타르틴베이커리
3324 	|을지로통일집
3325 	|곱창역
3326 	|병천유황오리
3327 	|쿠이신보
3328 	|포차더쎈
3329 	|동북양꼬치
3330 	|근린커피
3331 	|포도집
3332 	|녹스
3333 	|신숙
3334 	|미트스타킴
3335 	|아이엠베이글
3336 	|안선생주가
3337 	|나폴레옹과자점
3338 	|이태리로간고등어
3339 	|주막
3340 	|뭄알로이
3341 	|진진만두국
3342 	|초와밥
3343 	|닥터로빈
3344 	|낙서파전
3345 	|오레노라멘
3346 	|한방숯불바베큐
3347 	|라크라센타
3348 	|치어스
3349 	|오덕진해물능이버섯한방오리
3350 	|라고파스타
3351 	|라세느
3352 	|한나식빵
3353 	|paul310
3354 	|마구로
3355 	|제형면옥
3356 	|닭진미강원집
3357 	|형제상회
3358 	|쌍문동판다쓰
3359 	|미시락칼국수
3360 	|콩플레
3361 	|하우돈곱창
3362 	|카페몽글
3363 	|춘자싸롱
3364 	|망원동티라미수
3365 	|컴플리트커피


3930 	|개성손만두
3931 	|은행골
3932 	|심가네전골도둑
3933 	|침사추이누들
3934 	|이야초밥
3935 	|란주칼면
3936 	|매화반점
3937 	|33마켓
3938 	|경춘자의라면땡기는날
3939 	|빕스
3940 	|만선
3941 	|최가커피
3942 	|시키카츠
3943 	|토마스 피
3944 	|반짝반짝빛나는
3945 	|맛이차이나
3946 	|쿄우
3947 	|곱창의전설
3948 	|산들해
3949 	|테이스티마켓
3950 	|포베이
3951 	|치즈팩토리
3952 	|낙원타코
3953 	|오스틴
3954 	|삼양동순대떡볶이
3955 	|코코이찌방야
3956 	|장스테이크하우스
3957 	|인왕산대충유원지
3958 	|육회본가
3959 	|하쿠시
3960 	|리히트케이크
3961 	|도미어묵
3962 	|지구당
3963 	|바르미 샤브샤브
3964 	|스시와라
3965 	|벨르보
3966 	|뚱보롱
3967 	|한상차림밥상
3968 	|옐로우코코넛
3969 	|갯마을조개
3970 	|경천애인
3971 	|토끼네부엌
3972 	|연희김밥
3973 	|종로할머니맥주
3974 	|톡톡
3975 	|리얼후라이
3976 	|모두랑즉석떡볶이
3977 	|여기가좋겠네
3978 	|일번지육개장
3979 	|신세계떡볶이
3980 	|홍릉각
3981 	|북촌삼대갈비
3982 	|롤링핀
3983 	|구로곱창
3984 	|텐카이치
3985 	|정돈
3986 	|동대문털보네
3987 	|설빙
3988 	|72420
3989 	|송월냉면
3990 	|굿모닝통밀
3991 	|산채보리밥
3992 	|묵은지사랑
3993 	|정애맛담
3994 	|연탄불생고기
3995 	|장가네곱창
3996 	|바틸트
3997 	|대성집
3998 	|숙달돼지
3999 	|모모야마
4000 	|고기식당
4001 	|문래돼지불백
4002 	|카페 블리스
4003 	|만석곱창구이
4004 	|커피생각
4005 	|만희네참숯불닭갈비
4006 	|할머니냉면집
4007 	|에프스토리
40

In [20]:
M_restaurants_df

,name,address,category,main_mn,price,opng_tm,rating,rvw_cnt,tags
0,제이든,서울특별시 강서구 마곡중앙8로 90 마곡아이파크 217호,카페 / 디저트,None,None,10:00 - 22:00,NaN,5,None
1,본쁠리,서울특별시 강북구 오패산로30길 73 1F,카페 / 디저트,None,None,월-목: 12:00 - 22:00 금-토: 12:00 - 23:00,NaN,2,None
2,카페 달력,서울특별시 은평구 연서로18길 28-2,카페 / 디저트,None,None,10:00 - 22:00,3.7,8,None
3,라 쿠치나,서울특별시 용산구 회나무로44길 10,이탈리안,None,None,12:00 - 22:00,4.3,20,"이탈리안,콜키지가능,발렛가능,"
4,사월식당,서울특별시 광진구 자양로4길 12-1,회 / 스시,None,None,18:00 - 01:00,4,25,"술집추천,"
5,천진분식,서울특별시 서대문구 이화여대5길 7,딤섬 / 만두,None,None,None,4.2,16,"2020망고플레이트인기맛집,"
6,EDEN,서울특별시 송파구 백제고분로7길 24-13 201호,카페 / 디저트,None,None,11:00 - 24:00,3.7,8,None
7,수정불막창,서울특별시 은평구 연서로29길 11-9,고기 요리,None,None,16:00 - 24:00,None,9,"주먹밥,막창,특별한세상,계란찜,곱창,숯불,"
8,몬스터플레이스,서울특별시 용산구 청파로43길 70,기타 양식,None,None,None,4.3,18,"2020망고플레이트인기맛집,"
9,카페미르,서울특별시 강북구 한천로 988,카페 / 디저트,None,None,None,None,1,None


In [21]:
M_review_df

,res_name,user_name,rating,review
0,제이든,녹차가젤조아,3,9500원 이름은 까먹음혼자갔는데 원래 와플과 음료를 함께 시켜야한단다비싸지만 맛있...
1,제이든,냠냠,5,맛있어요! 밀크티가 두 종류 이상 있었고 (대만식? 이런 이름으로 있었는데 기억이 ...
2,제이든,머그컵,3,와플은 바삭바삭합니당 와플종류는 다양한데 재료가 떨어졌거나 토핑이 계절과일이라 주문...
3,제이든,토깽이,5,와플 맛있어요!!! 종류도 많아서 고르기 힘들음 ㅠㅠ크럼블 머시기 먹었는데 가격이 ...
4,제이든,Hot_duckku,5,와플맛집 여기 대만밀크티인가? 무튼 동남아쪽 지역 달아놓은 밀크티 갠찮습니다 아직이...
5,본쁠리,ayay,5,"미아사거리카페 카페,포레(처음엔 본쁠리 였던것 같다.)미아사거리에서 7분? 정도 걸..."
6,본쁠리,kakerlaken,5,"미아동에 위치한 작은 카페.아늑한 분위기는 물론이고,함께 운영하는 가죽공방이 멋스러..."
7,카페 달력,쫑크,3,#아무리뷰주거권이 있는 동네에 하나쯤 있어야 할 아기자기한 카페. 사실 어디에선가 ...
8,카페 달력,GABY,5,따뜻하고 아늑한 느낌의 카페-생각했던 것보다 날씨가 너무 추워서 덜덜 떨면서 문을 ...
9,카페 달력,DCT,5,안그래도 예뻐서 예전부터 와보고싶던 곳이었는데 심지어 애견동반까지 가능해서❣️ 강아...


In [22]:
M_restaurants_df.to_csv('../data/MangoPlate_df.csv',index=False)
M_review_df.to_csv('../data/MangoPlate_review_df.csv', index=False)